In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from datetime import datetime

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_climate = os.path.join(folder_AGVA, "Climate")

In [3]:
# define regions, summer months, winter months
regions = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
months_s = [6,7,8]
months_w = [1,2,3]

# list to save dicctionaries in
all_data = []

# for each subregion, open the climate csv
for r in regions:
    path_open = os.path.join(folder_climate, "O3Regions", f'Region_{str(r).zfill(2)}_monthly.csv')
    
    df_r = pd.read_csv(path_open)

    # for each year
    for y in [2018,2019,2020,2021,2022]:
        
        # subset to these years
        df_r_subset = df_r[df_r['year']==y]
        
        # get winter and summer subset
        df_r_subset_w = df_r_subset[df_r_subset['month'].isin(months_w)]
        df_r_subset_s = df_r_subset[df_r_subset['month'].isin(months_s)]
        
        # calculate the average temperature in this summer, winter
        t_w = np.nanmean(df_r_subset_w['temperature_2m'])
        t_s = np.nanmean(df_r_subset_s['temperature_2m'])
        
        # calculate the total precip in this summer, winter
        p_w = np.nansum(df_r_subset_w['total_precipitation_sum'])
        p_s = np.nansum(df_r_subset_s['total_precipitation_sum'])
        
        # calculate the total snow in this summer, winter
        s_w = np.nansum(df_r_subset_w['snowfall_sum'])
        s_s = np.nansum(df_r_subset_s['snowfall_sum'])
        
        # create dictionary to save data
        out_dic = {"O3Region":r, "year":y, 
                   'temperature_2m_s':t_s, 'temperature_2m_w':t_w, 
                   'total_precipitation_sum_s':p_s, 'total_precipitation_sum_w':p_w, 
                   'snowfall_sum_s':s_s, 'snowfall_sum_w':s_w, 
                  }
        all_data.append(out_dic)

In [4]:
# create df from the data
df_all = pd.DataFrame(all_data)

# group by region, calculate average
df_averages = df_all.groupby('O3Region').mean().reset_index()

# define all the column names
all_names = ["temperature_2m_s", "temperature_2m_w", "total_precipitation_sum_s", "total_precipitation_sum_w", "snowfall_sum_s", "snowfall_sum_w"]

# for each column, calculate each region/year's anomaly
all_anoms = df_all[['O3Region', 'year']].copy()
for n in all_names:
    subset = df_all[['O3Region', 'year', n]]
    subset = subset.merge(df_averages[['O3Region', n]], on=['O3Region'])
    
    # calculate temperature anomaly as magnitude
    if n[:4] == 'temp':
        all_anoms[f"{n}_anom"] = subset[f"{n}_x"]-subset[f"{n}_y"]
    
    # calculate precipitation anomaly as percent
    else:
        all_anoms[f"{n}_anom"] = (subset[f"{n}_x"]-subset[f"{n}_y"])/subset[f"{n}_y"]

# lets convert this to wide format. one row for each region
all_dics = []
for r in regions:
    subset = all_anoms[all_anoms['O3Region']==r]
    out_dic = {"O3Region":r}
    
    for y in [2018,2019,2020,2021,2022]:
        
        row = subset[subset['year']==y]
        for n in all_names: 
            out_dic[f'{n}_anom_{y}'] = row[f"{n}_anom"].values[0]
    all_dics.append(out_dic)

all_anoms_wide = pd.DataFrame(all_dics)
all_anoms

,O3Region,year,temperature_2m_s_anom,temperature_2m_w_anom,total_precipitation_sum_s_anom,total_precipitation_sum_w_anom,snowfall_sum_s_anom,snowfall_sum_w_anom
0,1,2018,-0.707480,0.660416,0.295256,0.001689,-0.690267,0.001626
1,1,2019,1.734656,2.606717,-0.386540,0.186485,-0.999306,0.003428
2,1,2020,0.059283,-3.354512,-0.008583,-0.292870,-0.351016,-0.168739
3,1,2021,-0.809415,0.089332,-0.037479,0.012378,1.777100,0.155175
4,1,2022,-0.277044,-0.001954,0.137346,0.092317,0.263489,0.008511
...,...,...,...,...,...,...,...,...
75,16,2018,0.691627,-0.512981,-0.434669,-0.337037,-0.488140,-0.340967
76,16,2019,0.206588,-0.719993,0.108684,-0.394624,1.272761,-0.447345
77,16,2020,-2.026668,-0.603237,0.555240,0.094216,0.561267,0.127909
78,16,2021,0.260341,0.094295,-0.104416,0.352368,-0.595298,0.394812


In [5]:
# save the anomaly dataset
path_out = os.path.join(folder_climate, "O3Regions", f'O3Region_anomalies.csv')
all_anoms_wide.to_csv(path_out, index=False)